# Перед началом работы

1. инструкция по установке

https://www.tensorflow.org/install/

Полезные ссылки:

https://web.stanford.edu/class/cs20si/syllabus.html

https://www.tensorflow.org/programmers_guide/

# Приступим

In [1]:
import tensorflow as tf

попробуем сложить два числа

In [40]:
a = tf.add(1,2)

и посмотрим результат

In [41]:
print (a)

Tensor("Add_3:0", shape=(), dtype=int32)


в результате мы получам не число, а ссылку на узел в графе вычеслений

Все переменные вычисляются в сессии - и только в том случае, если вы указали, что хотите посчитать их. До вычисления переменных только граф вычислений. Промежуточные результаты не вычисляются

In [42]:
sess = tf.Session()
a_res = sess.run(a)
sess.close()

посмотрим вычесленное значение узла графа

In [43]:
print (a_res)

3


не пишите так!! вы заменяете ссылку на узел графа вычислений на число.

In [44]:
with tf.Session() as sess:
    a = sess.run(a)
#     a_res = sess.run(a)
    print (a)

3


Построим более сложный граф вычислений и посмотрим на него в TensorBoard

# TensorBoard

в TensorFlow по умолчанию используется дефолтный граф вычислений. Добавим в него более сложный подграф и сохраним его содержимое в папку ./graphs.

In [2]:
x = 2
y = 3
add_op = tf.add(x, y)
mul_op = tf.multiply(x, y)
useless = tf.multiply(x, add_op)
pow_op = tf.pow(add_op, mul_op)

writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())

Посмотрим с помощью следующей команды, запущенной из консоли: 

tensorboard --logdir="./graphs" --port 6006

Далее открываем localhost:6006

И смотрим:)

![title](./pictures_for_tf_sem/pic1.png )

Можно заметить, что названия вершин генерируются автоматически исходя из типов операций в этой вершине. Для лучшей читаемости графа лучше называть переменные самому. В примеру именеим слоя: input, conv, linear и т.п.

In [2]:
a = tf.constant(2, name='a')
b = tf.constant(3, name='b')
x = tf.add(a, b, name='add')

x = tf.constant(2, name='input_word')
y = tf.constant(2, name='input_char')
add_op = tf.add(x, y)
mul_op = tf.multiply(x, y)
useless = tf.multiply(x, add_op)
pow_op = tf.pow(add_op, mul_op, name='output')

writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())

![title](./pictures_for_tf_sem/pic2.png )

# Variables


В TensorFlow вы имеете дело со следующими типами переменных

In [9]:
a = tf.constant([1, 2], name='a')
b = tf.constant([[0, 1], [2, 3]], name='b')

In [10]:
s = tf.Variable(2, name="scalar")
m = tf.Variable([[0, 1], [2, 3]], name="matrix") 
W = tf.Variable(tf.zeros([784,10]))
W_constant = tf.Variable(tf.zeros([784,10]), trainable=False)

Для создания изменяемых переменных используйте get_variable. Она имеет доступ в пространству имен и не позволит созвать вам уже существующую переменную с таким именем.

In [11]:
s = tf.get_variable("scalar", initializer=tf.constant(2))
m = tf.get_variable("matrix", initializer=tf.constant([[0, 1], [2, 3]]))
W = tf.get_variable("big_matrix", shape=(784, 10), initializer=tf.zeros_initializer())

tf.Variable - это класс со множеством доступных операций

x = tf.Variable(...) 

x.initializer # init op

x.value() # read op

x.assign(...) # write op

x.assign_add(...) # and more

In [12]:
scalar = tf.Variable(2, name="scalar")
one_more_scalar = tf.Variable(2, name="scalar")
scalar.name, one_more_scalar.name

('scalar_2:0', 'scalar_3:0')

In [13]:
one_more_scalar = tf.get_variable(initializer=tf.constant(2), name="scalar")

ValueError: Variable scalar already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\dima\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1654, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\Users\dima\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3290, in create_op
    op_def=op_def)
  File "C:\Users\dima\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)


In [14]:
v = tf.get_variable("vector", initializer=tf.constant([0, 1, 3]))
yav = tf.get_variable("yet_anoter_one_vector", initializer=tf.constant([4,5,6]))

In [ ]:
def linear(vec):
    w = tf.Variable(tf.random_normal((3,3)), name='weights')
    b = tf.Variable(tf.zeros((3,)), name='biases')
    return tf.matmul(vec, w) + b

linear1 = linear(v)
linear2 = linear(yav)

In [ ]:
def linear(vec):
    w = tf.get_variable("weights", (3,3), initializer=tf.random_normal_initializer())
    b = tf.get_variable("biases", (3,), initializer=tf.constant_initializer(0.0))
    return tf.matmul(vec, w) + b
linear1 = linear(v)
linear2 = linear(yav)

# Интерактив  
## В чем разница между 2 последними окошками?

In [ ]:
def linear(vec):
    w = tf.get_variable("weights", (3,3), initializer=tf.random_normal_initializer())
    b = tf.get_variable("biases", (3,), initializer=tf.constant_initializer(0.0))
    return tf.matmul(vec, w) + b

with tf.variable_scope('two_linears') as scope:
    linear1 = linear(v)
    scope.reuse_variables()
    linear2 = linear(yav)

In [ ]:
def linear(vec):
    w = tf.get_variable("weights", (3,3), initializer=tf.random_normal_initializer())
    b = tf.get_variable("biases", (3,), initializer=tf.constant_initializer(0.0))
    return tf.matmul(vec, w) + b

with tf.variable_scope('linear1', reuse=tf.AUTO_REUSE) as scope:
    linear1 = linear(v)
with tf.variable_scope('linear2', reuse=tf.AUTO_REUSE) as scope:
    linear2 = linear(yav)

## А теперь?

https://docs.google.com/presentation/d/1SJMsa4BdOFVRCPD9uwaAqDBYYfgbQcbOm7HaxRVpaaY/edit

## Разница между var_scope и name_scope

In [36]:
with tf.name_scope("my_scope"):
    v1 = tf.get_variable("var1", [1], dtype=tf.float32)
    v2 = tf.Variable(1, name="var2", dtype=tf.float32)
    a = tf.add(v1, v2)

print(v1.name)  # var1:0
print(v2.name)  # my_scope/var2:0
print(a.name)   # my_scope/Add:0

var1:0
my_scope/var2:0
my_scope/Add:0


In [37]:
with tf.variable_scope("my_scope"):
    v1 = tf.get_variable("var1", [1], dtype=tf.float32)
    v2 = tf.Variable(1, name="var2", dtype=tf.float32)
    a = tf.add(v1, v2)

print(v1.name)  # my_scope/var1:0
print(v2.name)  # my_scope/var2:0
print(a.name)   # my_scope/Add:0

my_scope/var1:0
my_scope_1/var2:0
my_scope_1/Add:0


In [38]:
with tf.name_scope("foo"):
    with tf.variable_scope("var_scope"):
        v = tf.get_variable("var", [1])
with tf.name_scope("bar"):
    with tf.variable_scope("var_scope", reuse=True):
        v1 = tf.get_variable("var", [1])
assert v1 == v
print(v.name)   # var_scope/var:0
print(v1.name)  # var_scope/var:0

var_scope/var:0
var_scope/var:0


взято с https://stackoverflow.com/questions/35919020/whats-the-difference-of-name-scope-and-a-variable-scope-in-tensorflow

Перед вычислением нужно инициализировать переменные заданными значениями

In [35]:
with tf.Session() as sess:
    print(sess.run(W))

FailedPreconditionError: Attempting to use uninitialized value big_matrix
	 [[Node: _retval_big_matrix_0_0 = _Retval[T=DT_FLOAT, index=0, _device="/job:localhost/replica:0/task:0/device:CPU:0"](big_matrix)]]

Можно это сделать любым из трех приведённых ниже способов

In [12]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(W))
    
# with tf.Session() as sess:
#     sess.run(tf.variables_initializer([a, b]))
    
# W = tf.Variable(tf.zeros([784,10]))
# with tf.Session() as sess:
#     sess.run(W.initializer)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


Вычилить значение переменной можно с помощью функции eval

In [13]:
W = tf.Variable(tf.zeros([784,10]))
with tf.Session() as sess:
    sess.run(W.initializer)
    print(W)
    print (W.eval())


<tf.Variable 'Variable_2:0' shape=(784, 10) dtype=float32_ref>
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


Также следите, какие именно узлы графа вы исполняете

In [14]:
s = tf.Variable(2, name="scalar")
s.assign(100)
with tf.Session() as sess:
    sess.run(s.initializer)
    print(s.eval())

2


In [15]:
s = tf.Variable(2, name="scalar")
s_op = s.assign(100)
with tf.Session() as sess:
    sess.run(s.initializer)
    sess.run(s_op)
    print(s.eval())

100


# Placeholder

При обучении мы каждый раз подаем новые значения входных данных и меток классов. Существует специальная переменная, в которую можно подавать на вход данные, которую не нужно инициализировать.

In [16]:
# create a placeholder for a vector of 3 elements, type tf.float32
a = tf.placeholder(tf.float32, shape=[3])

b = tf.constant([5, 5, 5], tf.float32)

# use the placeholder as you would a constant or a variable
c = a + b  # short for tf.add(a, b)

# with tf.Session() as sess:
#     print(sess.run(c))
    
    
with tf.Session() as sess:
    print(sess.run(c, {a: [2, 2, 3]}))


[ 7.  7.  8.]


# IF

if в при создании графа вычислений должен зависеть от параметров, не меняющихся в процессе вычисления графа.

# Lazy loading

Выносите все в специальные узлы графа

In [17]:
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')
z = tf.add(x, y)

writer = tf.summary.FileWriter('./graphs/normal_loading', tf.get_default_graph())
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(10):
        sess.run(z)

Обратите внимание: код ниже написан короче, но создает 10 лишних вершин в графе.

In [18]:
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')

writer = tf.summary.FileWriter('./graphs/normal_loading', tf.get_default_graph())
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(10):
        sess.run(tf.add(x, y))

# Simple network

In [17]:
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

Определить вид растения(Щестинистый, разноцветный, виргинский) по данным измерения  
Длина наружной доли околоцветника (sepal length)  
Ширина наружной доли околоцветника (sepal width)  
Длина внутренней доли околоцветника (petal length)  
Ширина внутренней доли околоцветника (petal width)  

![alt text](https://upload.wikimedia.org/wikipedia/commons/5/56/Iris_dataset_scatterplot.svg "Iris Text 1")

picture from wikipedia

In [18]:
# load iris dataset (Ирисы Фишера)


TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    return train_path, test_path

def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
    train_path, test_path = maybe_download()

    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y), (test_x, test_y)

In [19]:
(train_x, train_y), (test_x, test_y) = load_data()

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [20]:
# create generator
def next_batch(X_data, Y_data, batch_size=10):
    length = X_data.shape[0]
    reduced_length = (length // batch_size) * batch_size
    
    X_data = X_data[:reduced_length].reshape([batch_size, -1, 4])
    Y_data = Y_data[:reduced_length].reshape([batch_size, -1]) 
    
    X_data = np.transpose(X_data, axes=(1, 0, 2))
    Y_data = np.transpose(Y_data, axes=(1, 0))
    for x_i, y_i in zip(X_data, Y_data):
        yield x_i, y_i

In [21]:
# create placeholders
batch_size = 10

#X = tf.placeholder(tf.float32, shape=[batch_size, 4], name = 'input')
#Y = tf.placeholder(tf.int32, shape=[batch_size], name = 'output')
X = tf.placeholder(tf.float32, shape=[None, 4], name = 'input')
Y = tf.placeholder(tf.int32, shape=[None], name = 'output')

In [22]:
# create simple network

layer_size = 50

# initialization - http://cs231n.github.io/neural-networks-2/#init
initializer = tf.random_normal_initializer(stddev=np.sqrt(2/layer_size), mean=0.0)

# create weight and bias
with tf.variable_scope(name_or_scope='layer_1'):
    w1 = tf.get_variable('weights', shape=[4,layer_size], initializer=initializer)
    b1 = tf.get_variable('bias', shape=[layer_size], initializer=initializer)
    output_from_layer = tf.nn.relu(tf.matmul(X, w1) + b1)

    
with tf.variable_scope(name_or_scope='layer_2'):
    w2 = tf.get_variable('weights', shape=[layer_size,3], initializer=initializer)
    b2 = tf.get_variable('bias', shape=[3], initializer=initializer)
    Y_predicted = tf.matmul(output_from_layer, w2) + b2

In [23]:
# use tf.reduce_mean for batches
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits = Y_predicted, labels = Y), name='loss')

In [24]:
# choose optimizer to minimize loss
optimizer = tf.train.AdamOptimizer().minimize(loss)

# TAKE SUMMARY

In [25]:
# make summaries
tf.summary.scalar('loss', loss)
tf.summary.histogram('weights1', w1)
tf.summary.histogram('bias1', b1)

merged = tf.summary.merge_all()

In [26]:
# for name of summaries

from datetime import datetime
def time_str():
    return datetime.now().replace(microsecond=0).isoformat().replace(':','-')

In [27]:
start = time.time()
with tf.Session() as sess:
    # initialize the necessary variables
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('./summuries%s' %time_str(), sess.graph)
    
    # train the model for 100 epochs
    for i in range(100):
        total_loss = 0
        for x_i, y_i in next_batch(train_x.values, train_y.values):
            summary, _, l = sess.run(
                [merged,
                 optimizer, 
                 loss],feed_dict= 
                {X:x_i, 
                 Y:y_i}) 
            
            total_loss += l  
        train_writer.add_summary(summary, global_step=i)
        if (i + 1) % 10 == 0:
            print('Epoch {0}: {1}'.format(i, total_loss))

Epoch 9: 8.166786253452301
Epoch 19: 5.675806969404221
Epoch 29: 4.412605345249176
Epoch 39: 3.5488267093896866
Epoch 49: 2.901623919606209
Epoch 59: 2.4162718653678894
Epoch 69: 2.0557584688067436
Epoch 79: 1.787713535130024
Epoch 89: 1.5863817557692528
Epoch 99: 1.432706505060196


see summaries in TensorBoard

for example: tensorboard --logdir="./iris/summuries2018-03-29T14-08-35/" --port 6006

In [28]:
# test our model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    print ('test' + '--'*20)
    print ('predicted -- true')

    for x_i, y_i in next_batch(test_x.values, test_y.values):
        Y_pred = sess.run(
            [Y_predicted],
            feed_dict= 
            {X:x_i})
        for index_i, Y_in_batch in enumerate(Y_pred[0]):
#             print (Y_in_batch)
            print ('{0} -- {1}'.format(np.argsort(Y_in_batch)[-1], y_i[index_i]))

test----------------------------------------
predicted -- true
1 -- 1
1 -- 1
1 -- 0
1 -- 2
1 -- 2
1 -- 0
1 -- 0
1 -- 1
1 -- 1
1 -- 1
1 -- 2
1 -- 1
1 -- 2
1 -- 2
1 -- 1
1 -- 1
1 -- 2
1 -- 2
1 -- 1
1 -- 2
1 -- 0
1 -- 1
1 -- 1
1 -- 0
1 -- 1
1 -- 0
1 -- 0
1 -- 1
1 -- 0
1 -- 1


In [29]:
# why so bad results?
# we test a network with random value in layers! because we initialized it again

In [30]:
start = time.time()
with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    
    for i in range(100):
        total_loss = 0
        for x_i, y_i in next_batch(train_x.values, train_y.values):
            _, l = sess.run(
                [optimizer, 
                 loss],
                feed_dict= 
                {X:x_i, 
                 Y:y_i}) 
            
            total_loss += l  
        if (i + 1) % 10 == 0:
            print('Epoch {0}: {1}'.format(i, total_loss))
            
    print ('test' + '--'*20)
    print ('predicted -- true')

    for x_i, y_i in next_batch(test_x.values, test_y.values):
        Y_pred = sess.run(
            [Y_predicted],
            feed_dict= 
            {X:x_i})
        for index_i, Y_in_batch in enumerate(Y_pred[0]):
#             print (Y_in_batch)
            print ('{0} -- {1}'.format(np.argsort(Y_in_batch)[-1], y_i[index_i]))

print('Took: %f seconds' %(time.time() - start))

Epoch 9: 7.990514516830444
Epoch 19: 5.279015392065048
Epoch 29: 4.149567157030106
Epoch 39: 3.3766963183879852
Epoch 49: 2.789305418729782
Epoch 59: 2.3463463485240936
Epoch 69: 2.0151787772774696
Epoch 79: 1.7676478698849678
Epoch 89: 1.580244705080986
Epoch 99: 1.4356933198869228
test----------------------------------------
predicted -- true
1 -- 1
1 -- 1
0 -- 0
2 -- 2
2 -- 2
0 -- 0
0 -- 0
1 -- 1
1 -- 1
1 -- 1
2 -- 2
1 -- 1
1 -- 2
2 -- 2
1 -- 1
1 -- 1
2 -- 2
2 -- 2
1 -- 1
2 -- 2
0 -- 0
1 -- 1
1 -- 1
0 -- 0
1 -- 1
0 -- 0
0 -- 0
1 -- 1
0 -- 0
1 -- 1
Took: 0.743571 seconds


# Save  and restore model

tf.train.Saver.save(sess, save_path, global_step=None...)


tf.train.Saver.restore(sess, save_path)

In [31]:
saver = tf.train.Saver(max_to_keep=50)
# saver = tf.train.Saver({'variable': v1, 'v2': v2})

In [33]:
start = time.time()
with tf.Session() as sess:
    # Step 7: initialize the necessary variables, in this case, w and b
    sess.run(tf.global_variables_initializer())
    
    # Step 8: train the model for 100 epochs
    for i in range(100):
        total_loss = 0
        for x_i, y_i in next_batch(train_x.values, train_y.values, batch_size=10):
            _, l = sess.run(
                [optimizer, 
                 loss], 
                {X:x_i, 
                 Y:y_i}) 
            
            total_loss += l  
        if (i + 1) % 5 == 0:
            print('Epoch {0}: {1}'.format(i, total_loss))
            saver.save(sess, 
            './model',
            global_step=i)


print('Took: %f seconds' %(time.time() - start))

Epoch 4: 10.408232390880585
Epoch 9: 7.42070198059082
Epoch 14: 6.0516665279865265
Epoch 19: 5.220244735479355
Epoch 24: 4.623729318380356
Epoch 29: 4.143933460116386
Epoch 34: 3.738716259598732
Epoch 39: 3.3843933939933777
Epoch 44: 3.0637393295764923
Epoch 49: 2.791569545865059
Epoch 54: 2.5571112260222435
Epoch 59: 2.354011930525303
Epoch 64: 2.178238794207573
Epoch 69: 2.026128336787224
Epoch 74: 1.8946508169174194
Epoch 79: 1.7808574810624123
Epoch 84: 1.6820202842354774
Epoch 89: 1.5960067920386791
Epoch 94: 1.520504105836153
Epoch 99: 1.454014390707016
Took: 8.624159 seconds


In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, './iris/model_network/model-4')
    
    print ('test' + '--'*20)
    print ('predicted -- true')

    for x_i, y_i in next_batch(test_x.values, test_y.values):
        Y_pred = sess.run(
            [Y_predicted],
            feed_dict= 
            {X:x_i})
        for index_i, Y_in_batch in enumerate(Y_pred[0]):
#             print (Y_in_batch)
            print ('{0} -- {1}'.format(np.argsort(Y_in_batch)[-1], y_i[index_i]))

INFO:tensorflow:Restoring parameters from ./iris/model_network/model-4
